In [1]:
# Load Libraries
import numpy as np # linear algebra
import pandas as pd # dataframes
import matplotlib.pyplot as plt # General visualisations
import matplotlib.ticker as mtick # Axis visuals
import seaborn as sns # Statistical visualisations
import statsmodels.api as sm # Linear Regression

In [2]:
# load data into pandas dataframe
df_customers = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv")
df_orders = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv")
df_payments = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv")
df_items = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv")
df_products = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv")

In [3]:
dfs = {
    'customers': df_customers,
    'orders': df_orders,
    'payments': df_payments,
    'items': df_items,
    'products':df_products
}

In [4]:
# show data shapes of each dataset and missing value 
for ds, df in dfs.items():
    print('{}: {} x {}'.format(ds,df.shape[0], df.shape[1]))
    print(df.isnull().sum()/df.count()*100)
    print('\n')
print('\n')

customers: 99441 x 5
customer_id                 0.0
customer_unique_id          0.0
customer_zip_code_prefix    0.0
customer_city               0.0
customer_state              0.0
dtype: float64


orders: 99441 x 8
order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_approved_at                0.161159
order_delivered_carrier_date     1.825759
order_delivered_customer_date    3.073303
order_estimated_delivery_date    0.000000
dtype: float64


payments: 103886 x 5
order_id                0.0
payment_sequential      0.0
payment_type            0.0
payment_installments    0.0
payment_value           0.0
dtype: float64


items: 112650 x 7
order_id               0.0
order_item_id          0.0
product_id             0.0
seller_id              0.0
shipping_limit_date    0.0
price                  0.0
freight_value          0.0
dtype: float64


products: 32951 x 9
product_i

In [5]:
# describe each dataset
for ds, df in dfs.items():
    print('{}: {} x {}'.format(ds,df.shape[0], df.shape[1]))
    #print(df.describe())
    print('\n')
print('\n')

customers: 99441 x 5


orders: 99441 x 8


payments: 103886 x 5


items: 112650 x 7


products: 32951 x 9






# 2. Preparing Data

## 2.1 order level aggregation

In [6]:
# Merge order level data into one dataframe

# Left join with payments data (1 to 1)
dfm_orders = pd.merge(dfs['orders'], dfs['payments'],on="order_id",how="left")\
    .loc[:, ['customer_id', 'order_id','order_purchase_timestamp', 'payment_type','payment_value']]

# Left join with items data ( 1 to N)
dfm_orders = dfm_orders.merge(dfs['items'],on="order_id",how="left")\
    .loc[:, dfm_orders.columns.to_list() + ['order_item_id','product_id','seller_id', 'price', 'freight_value']]

# Left join with products data ( 1 to 1)
dfm_orders = dfm_orders.merge(dfs['products'],on="product_id",how="left")\
    .loc[:, dfm_orders.columns.to_list() + ['product_category_name']]

# add derived purchase date info (year, month also available)
dfm_orders.insert(dfm_orders.shape[1],
                    'order_purchase_date',
                    pd.to_datetime(dfm_orders.order_purchase_timestamp).dt.date
                   )

In [7]:
dfm_orders.shape

(118434, 12)

In [8]:
# check missing data
dfm_orders.isnull().sum()

customer_id                    0
order_id                       0
order_purchase_timestamp       0
payment_type                   3
payment_value                  3
order_item_id                830
product_id                   830
seller_id                    830
price                        830
freight_value                830
product_category_name       2528
order_purchase_date            0
dtype: int64

In [9]:
# debug only
dfm_orders.groupby('payment_type').size()

payment_type
boleto         23037
credit_card    87286
debit_card      1698
not_defined        3
voucher         6407
dtype: int64

In [10]:
# debug only
dfm_orders[dfm_orders.order_purchase_date.isnull()]

,customer_id,order_id,order_purchase_timestamp,payment_type,payment_value,order_item_id,product_id,seller_id,price,freight_value,product_category_name,order_purchase_date


In [11]:
# fill missing data of payment_value
dfm_orders['payment_value'] = np.where(dfm_orders.payment_value.isnull(),dfm_orders.price+dfm_orders.freight_value, dfm_orders.payment_value)
dfm_orders['payment_type'] = np.where(dfm_orders.payment_type.isnull(),'credit_card',dfm_orders.payment_type)
dfm_orders['price'] = np.where(dfm_orders.price.isnull(),dfm_orders.payment_value, dfm_orders.price)
dfm_orders['freight_value'] = np.where(dfm_orders.freight_value.isnull(),0, dfm_orders.freight_value)
dfm_orders['product_category_name'] = np.where(dfm_orders.product_category_name.isnull(),'',dfm_orders.product_category_name)

In [12]:
agg_order = dfm_orders.groupby('order_id').agg({
 'customer_id':'max',
 'payment_value':'sum',
 'price':'sum',
 'freight_value':'sum',
 'order_item_id':'count',
 'product_category_name': lambda x: set(x),
 'payment_type': lambda x: set(x),
}).reset_index().rename(columns={
 'customer_id':'customer_id',
 'payment_value':'spend',
 'price':'price',
 'freight_value':'shipping_fee',
 'order_item_id':'basket_size',
 'product_category_name': 'product_categories',
 'payment_type': 'payment_types'
})

In [13]:
agg_order.head()

,order_id,customer_id,spend,price,shipping_fee,basket_size,product_categories,payment_types
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,72.19,58.90,13.29,1,{cool_stuff},{credit_card}
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,259.83,239.90,19.93,1,{pet_shop},{credit_card}
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,216.87,199.00,17.87,1,{moveis_decoracao},{credit_card}
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,25.78,12.99,12.79,1,{perfumaria},{credit_card}
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,218.04,199.90,18.14,1,{ferramentas_jardim},{credit_card}


In [14]:
# convert set to string
#agg_order.drop(columns=['payment_type_set','product_cat_set'])
agg_order['payment_types'] = agg_order['payment_types'].apply(lambda x: ','.join(x))
agg_order['product_categories'] = agg_order['product_categories'].apply(lambda x: ','.join(x))

In [15]:
agg_order.groupby('product_categories').size()

product_categories
                                                           2164
,automotivo                                                   1
,beleza_saude                                                 2
,brinquedos                                                   1
,cama_mesa_banho                                              6
                                                           ... 
utilidades_domesticas,construcao_ferramentas_construcao       1
utilidades_domesticas,dvds_blu_ray                            1
utilidades_domesticas,industria_comercio_e_negocios           1
utilidades_domesticas,informatica_acessorios                  4
utilidades_domesticas,livros_interesse_geral                  1
Length: 346, dtype: int64

In [16]:
agg_order.groupby('payment_types').size()

payment_types
boleto                    19784
credit_card               74260
debit_card                 1527
debit_card,credit_card        1
not_defined                   3
voucher                    1621
voucher,credit_card        2245
dtype: int64

In [17]:
agg_order.head()

,order_id,customer_id,spend,price,shipping_fee,basket_size,product_categories,payment_types
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,72.19,58.90,13.29,1,cool_stuff,credit_card
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,259.83,239.90,19.93,1,pet_shop,credit_card
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,216.87,199.00,17.87,1,moveis_decoracao,credit_card
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,25.78,12.99,12.79,1,perfumaria,credit_card
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,218.04,199.90,18.14,1,ferramentas_jardim,credit_card


## 2.2 build customer level modellig dataset

In [18]:
# aggregation at customer level
dfm_customer = pd.merge(agg_order, dfs['customers'], on='customer_id',how='inner')\
    .loc[:,agg_order.columns.to_list() + ['customer_unique_id','customer_state']]

agg_customer = dfm_customer.groupby('customer_unique_id').agg({
    'order_id':'count',
    'spend':'sum',
    'basket_size':'sum',
    'product_categories': lambda x: set(x),
    'payment_types': lambda x: set(x)
    
}).reset_index().rename(columns={
    'order_id':'orders',
    'spend':'total_spend',
    'basket_size':'basket_size',
    'product_categories': 'product_categories',
    'payment_types': 'payment_types'
})

In [19]:
# debug
#agg_customer.dtypes
agg_customer.head()

,customer_unique_id,orders,total_spend,basket_size,product_categories,payment_types
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,1,{cama_mesa_banho},{credit_card}
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,1,{beleza_saude},{credit_card}
2,0000f46a3911fa3c0805444483337064,1,86.22,1,{papelaria},{credit_card}
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,1,{telefonia},{credit_card}
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,1,{telefonia},{credit_card}


In [20]:
agg_customer.groupby('product_categories').size()

TypeError: unhashable type: 'set'

In [ ]:
# convert lists to categorical 
# agg_customer.drop(columns=['product_categories_set','payment_types_set'],inplace=True)
agg_customer['product_categories'] = agg_customer['product_categories'].apply(
    lambda x: ','.join(set(','.join(x).split(','))))
agg_customer['payment_types'] = agg_customer['payment_types'].apply(
    lambda x: ','.join(set(','.join(x).split(','))))

In [ ]:
#debug only
agg_customer.groupby('product_categories').size()

In [ ]:
#agg_customer['total_spend'].hist()
#sns.boxplot(x=agg_customer['total_spend'])
agg_customer['total_spend'].describe()

In [ ]:
# Normalize columns
#agg_customer['payment_types'] = agg_customer['payment_types'].apply(lambda x: x if len(x.split(','))==1 and x != 'not_defined' else ('credit_card,others' if 'credit_card_with_others' in x.split(',')  else 'others') )
#agg_customer['product_categories'] = agg_customer['product_categories'].apply(lambda x: x if len(x.split(','))==1 else 'multiple')
#agg_customer['basket_size'] = agg_customer['basket_size'].apply(lambda x: 8 if x >7 else x)
#agg_customer['orders'] = agg_customer['orders'].apply(lambda x: 3 if x >2 else x)
agg_customer['total_spend'] = agg_customer['total_spend'].apply(lambda x: 200 if x >=200 else x)

In [ ]:
# add dimension 
agg_customer = agg_customer.merge(dfs['customers'], on="customer_unique_id", how="inner")\
    .loc[:, agg_customer.columns.to_list() + ['customer_state']]

In [ ]:
# debug
agg_customer.head()

## 2.3 feature engineering for customer segmentation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# interactive view, dashboard
import plotly.express as px
import plotly.graph_objs as pgo

In [ ]:
# feature engineering
#get convert categorical variables to binary variables 
features = agg_customer.drop(columns=['customer_state'])
features = pd.get_dummies(data=features, columns=['payment_types','product_categories'])

In [ ]:
features.head()

In [ ]:
X = features.sample(n=6000, replace=False, random_state=1).drop(columns=['customer_unique_id'])

In [ ]:
scalar = StandardScaler()
Xs = scalar.fit_transform(X)

In [ ]:
# initial test with max components
pca = PCA(n_components=min(Xs.shape[0], Xs.shape[1]))
d = pca.fit_transform(Xs)

In [ ]:
# first the first N components that explain at least 80% of total variance
scree = pca.explained_variance_ratio_
nbr_pca=0
N=110
for i in range(N):
    a = scree.cumsum()[i]
    if a >= 0.80:
        print("{} principal components explain at least 80% of the total variance".format(i))
        print("Exact value of variance explained: {}%".format(round(a*100,2)))
        nbr_pca=i
        break

In [ ]:
# select component, re-engineering features
pca = PCA(n_components=50)
d=pca.fit_transform(Xs)

data_pca=pd.DataFrame(d)
col=["pca "+ str(n+1) for n in data_pca.columns]
data_pca.columns=col
data_pca.head()

In [ ]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}
fig = px.scatter_matrix(
    d,
    labels=labels,
    dimensions=range(3),
    color=X["basket_size"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    d, x=0, y=1, z=2, color=X["orders"],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

# 3. Clustering

## 3.1 K-Means

In [ ]:
# import library 

# K-Means clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_score, silhouette_samples

# plot
import matplotlib.cm as cm
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as pgo


In [ ]:
# use PCA selected features for modeling

X=data_pca.copy()
scaler=StandardScaler() 
Xs=scaler.fit_transform(X)

In [ ]:
# method 1: elbow method. 
# intertia is mean squared distance of instance and its closest centroid.
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = "k-means++", max_iter = 500, n_init = 10, random_state = 123)
    kmeans.fit(Xs)
    wcss.append(kmeans.inertia_)

fig = go.Figure(data = go.Scatter(x = [1,2,3,4,5,6,7,8,9,10], y = wcss))
fig.update_layout(title='WCSS vs. Cluster number', xaxis_title='Clusters', yaxis_title='WCSS')
fig.show()

In [ ]:
# method 2: silhouette score
# silhouette score meants the relative distance of instance from neighbor centroids (b) and its own centroid (a). formula: (b-a)/max(a,b). Close to +1 is better. 

n_clusters=range(2,20)

#n_clusters=[2,10,20,30,40,50]
scores=[]
for n in n_clusters:
    #print('{} clusters...'.format(n))
    kmeans = KMeans(n_clusters=n, random_state=0).fit(Xs)
    labs=kmeans.labels_
    score=silhouette_score(Xs,labs)
    scores.append(score)

In [ ]:
# plot silhouette_score 
fig = px.line(x=n_clusters, y=scores, title='silhouette_score')
fig.show()

In [ ]:
SEED = 42
def train_kmeans(X):
  #ks = np.linspace(2, 8, 7, dtype=np.int64)
  inertias = []
  silhouettes = []
  kmeans_k = []
  for k in ks:
    kmeans = KMeans(n_clusters=k, random_state=SEED)
    kmeans.fit(X)

    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X, kmeans.labels_))
    kmeans_k.append(kmeans)

  return kmeans_k, inertias, silhouettes, ks

In [ ]:
ks = np.linspace(2, 8, 7, dtype=np.int64)
kmeans_k, inertias, silhouettes = train_kmeans(Xs)

Viz reference
https://github.com/alexandrehsd/Cluster-Analysis/blob/master/Silhouette%20Analysis.ipynb

In [ ]:
# best number of clusters is 3
N=6
kmeans = KMeans(n_clusters=N, random_state=0).fit(Xs)
labs=kmeans.labels_

In [ ]:
# plot
test2d = data_pca.loc[:,["pca 1","pca 2"]]
test2d['cluster'] = labs
test3d = data_pca.loc[:,["pca 1","pca 2",'pca 3']]
test3d['cluster'] = labs

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.scatter(test2d, x="pca 1", y="pca 2", color='cluster', title='2D'),
    row=1, col=1
)

fig.add_trace(
    px.scatter_3d(test3d, x='pca 1', y='pca 2', z='pca 3', color='cluster', title='3D'),
    row=1, col=2
)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

In [ ]:
#2d plot
test2d = data_pca.loc[:,["pca 1","pca 2"]]
test2d['cluster'] = labs
fig = px.scatter(test2d, x="pca 1", y="pca 2", color='cluster')
fig.show()

In [ ]:
# 3d plot

test3d=data_pca.loc[:,["pca 1","pca 2","pca 3"]]
test3d["cluster"]=labs

fig = px.scatter_3d(
    test3d, x='pca 1', y='pca 2', z='pca 3', color='cluster',
    title=f'Number of clusters is 3',
)
fig.show()

In [ ]:
# method 3

range_n_clusters = [2,3,4,5,6]

fig, ax = plt.subplots(5, 2, figsize=(16, 20))
for row, n_clusters in enumerate(range_n_clusters):

  # The 1st subplot is the silhouette plot
  # The silhouette coefficient can range from -1, 1 but in this example all lie within [-0.1, 1]
  ax[row, 0].set_xlim([-0.1, 1])

  # The (n_clusters+1)*10 is for inserting blank space between silhouette
  # plots of individual clusters, to demarcate them clearly.
  ax[row, 0].set_ylim([0, len(Xs) + (n_clusters + 1) * 10])

  # get predictions for each label
  cluster_labels = kmeans_k[n_clusters-2].predict(Xs)

  # The silhouette_score gives the average value for all the samples.
  # This gives a perspective into the density and separation of the formed clusters
  silhouette_avg = silhouette_score(Xs, cluster_labels)
  print("For n_clusters =", n_clusters, ", the average silhouette_score is :", silhouette_avg)
  
  # Compute the silhouette score for each sample
  sample_silhouette_values = silhouette_samples(Xs, cluster_labels)

  y_lower = 10
  for i in range(n_clusters):
    # Aggregate the silhouette scores for samples belonging to # cluster i, and sort them
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = cm.nipy_spectral(float(i) / n_clusters)
    ax[row, 0].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values,facecolor=color, edgecolor=color, alpha=0.7)

    # Label the silhouette plots with their cluster numbers at the middle
    ax[row, 0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

  ax[row, 0].set_title("Silhouette diagram with = {}".format(n_clusters), fontsize=16)
  ax[row, 0].set_xlabel("Silhouette coefficient values", fontsize=14)
  ax[row, 0].set_ylabel("Cluster label", fontsize=12)
  
  ax[row, 0].set_yticks([])  # Clear the yaxis labels / ticks
  ax[row, 0].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
  ax[row, 0].tick_params(axis='both', which='major', labelsize=12)

  # The vertical line for average silhouette score of all the values
  ax[row, 0].axvline(x=silhouette_avg, color="black", linestyle="--", linewidth=2)

  ## right side of chart
  colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
  ax[row, 1].scatter(Xs[:, 0], Xs[:, 1], marker='.', s=30, lw=0, alpha=0.7, c=colors, edgecolor='k')
  
  # Labeling the clusters
  centers = kmeans_k[n_clusters-2].cluster_centers_
  # Draw white circles at cluster centers
  ax[row, 1].scatter(centers[:, 0], centers[:, 1], marker='o', c="white", alpha=1, s=200, edgecolor='k')
  
  for i, c in enumerate(centers):
    #ax[row, 1].scatter(c[0], c[1], marker=' '% i, alpha=1,s=50, edgecolor='k')
    ax[row, 1].scatter(c[0], c[1], marker='.', alpha=1,s=50, edgecolor='k')

  ax[row, 1].set_title(f"Clustered data with k = {n_clusters}", fontsize=16)
  ax[row, 1].set_xlabel("", fontsize=14)
  ax[row, 1].set_ylabel("", fontsize=14)
  ax[row, 1].set_yticks([]) 
  ax[row, 1].set_xticks([])

  ax[row, 1].grid(False)

  # fig.suptitle(("Silhouette analysis for KMeans clustering "with = %d" % n_clusters), fontsize=14, fontweight='bold')

plt.tight_layout()
#plt.savefig("silhouette_diagram.png", dpi=650, transparent=False)
plt.show()
     

# Hierarchical Clustering (TODO)

In [ ]:
labs